In [23]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import date
import time
import random
import pickle
import pandas as pd
import numpy as np
import re
import os

In [24]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [25]:
wait_time = 3
def loadDataframe():
    dataFrameEmpty = {
        'id': [],
        'title': [],
        'company': [],
        'location': [],
        "salary": [],
        'link': [],
        "retrieved_on": [],
        "post_data": [],
        "description": []
        
    }
    wait_time = 1
    df = pd.DataFrame(dataFrameEmpty)
    if os.path.exists('job_search_database.csv'):
        df = pd.read_csv('job_search_database.csv')
    return df


In [26]:
job_id_cache = {}
if os.path.exists('job_id_cache.pickle'):
    with open('job_id_cache.pickle', 'rb') as f:
        # Pickle the dictionary
        job_id_cache = pickle.load(f)

In [27]:
blackList = []
if os.path.exists("word_blacklist.json"):
    with open("word_blacklist.json", "r") as f:
        data = json.load(f)
        blackList = data["blacklist"]

def inBlackList(text):
    for elm in blackList:
        if re.search(elm, text.lower()) is not None:
            return False
    return True

In [28]:
def saveDictionary(name, dict):
    with open(name, 'wb') as f:
        pickle.dump(dict, f)

In [29]:
def get_element_text_by_classname(driver, class_name):
    found = driver.find_elements(By.CSS_SELECTOR, "." + class_name)
    if len(found) <= 0:
        return ""
    return found[0].text

In [30]:
def login(driver):
    driver.get("https://www.linkedin.com/login")
    time.sleep(wait_time)
    with open('credentials.json') as f:
        data = json.load(f)
        USERNAME = data['username']
        PASSWORD = data['password']
    
        username = driver.find_element(By.ID, "username")
        username.send_keys(USERNAME)
        time.sleep(wait_time)
        
        pword = driver.find_element(By.ID, "password")
        pword.send_keys(PASSWORD) 
    time.sleep(wait_time)
    
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    time.sleep(wait_time)

In [31]:
def getJobData(driver):
    job_posts = []
    resultList = driver.find_element(By.CSS_SELECTOR, ".scaffold-layout__list-container")
    jobs = resultList.find_elements(By.CSS_SELECTOR, ".jobs-search-results__list-item")
    for job in jobs:
        job_id = job.get_attribute('data-occludable-job-id')
        
        if (inBlackList(job.text) and job_id not in job_id_cache):
            job_id_cache[job_id] = True
            jobTitleContainerList = job.find_elements(By.CSS_SELECTOR, ".job-card-list__title")
            jobMetaData = job.find_elements(By.CSS_SELECTOR, ".job-card-container__metadata-item")
            posting_link = ""
            job_title = ""
            salary = ""
            location = ""
            today = date.today()
            if len(jobMetaData) > 1:
                location = jobMetaData[0].text
            if len(jobMetaData) > 2:
                salary = jobMetaData[1].text
            if len(jobTitleContainerList) > 0:
                posting_link = jobTitleContainerList[0].get_attribute('href')
                job_title = jobTitleContainerList[0].text
            company_name = get_element_text_by_classname(job, "job-card-container__primary-description")
            if job_title != "" and inBlackList(job_title):
                job_posts.append([job_id, job_title, company_name, location, salary, posting_link, today.strftime('%B %d, %Y')])
    return job_posts

In [32]:
def getJobDescriptions(driver, listOfJobs):
    newJobList = []
    for idx in range(len(listOfJobs)):
        job = listOfJobs[idx]
        print(job)
        driver.get(job[5])
        tvm_text = get_element_text_by_classname(driver, "tvm__text tvm__text--neutral")
        # click expand button
        driver.find_element(By.CSS_SELECTOR, ".jobs-description__footer-button").click()
        description = get_element_text_by_classname(driver, "jobs-description-content__text")
        if (inBlackList(description[17:])):
            newJobList.append(listOfJobs[idx] + [tvm_text] + [description[17:]])
        
        time.sleep(random.randint(0, 30))
        
    return newJobList


In [33]:
def openBrowserAndSearch():
    options = Options()
    options.add_argument('--ignore-certificate-errors')
    driver = uc.Chrome(options)

    login(driver)
    time.sleep(wait_time)
    # Go to job search page
    URL = "https://www.linkedin.com/jobs/search/?currentJobId=3822054670&f_E=2&f_TPR=r2592000&geoId=100017349&keywords=software%20engineer&location=80247%2C%20Denver%2C%20Colorado%2C%20United%20States&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true"
    driver.get(URL)
    time.sleep(wait_time)
    return driver

def main():
    df = loadDataframe()
    driver = openBrowserAndSearch()
    jobsOnPage = getJobData(driver)
    jobsOnPageWithDescription = getJobDescriptions(driver, jobsOnPage)
    for job in jobsOnPageWithDescription:
        df.loc[len(df.index)] = job
        
    df.to_csv("job_search_database.csv", sep=',', index=False, encoding='utf-8')
    saveDictionary("job_id_cache.pickle", job_id_cache)

In [16]:
main()

relocation
relocate
TS/SCI
clearance
secret


NameError: name 'job_id_cache' is not defined

In [34]:
df = loadDataframe()
driver = openBrowserAndSearch()
jobsOnPage = getJobData(driver)
jobsOnPageWithDescription = getJobDescriptions(driver, jobsOnPage)
for job in jobsOnPageWithDescription:
    df.loc[len(df.index)] = job
        
df.to_csv("job_search_database.csv", sep=',', index=False, encoding='utf-8')
saveDictionary("job_id_cache.pickle", job_id_cache)

['3843333593', 'Database/Web Engineer', 'Rincon Research Corporation', 'Centennial, CO (On-site)', '', 'https://www.linkedin.com/jobs/view/3843333593/?eBP=CwEAAAGOUz6zWSxY8JvhefAGscNFBSyLAAnbADdjQgp_84omb5LZ27CiVlc8_eZVjyllcWUgV_Rf1VFHx8dzUqAEOW6VYTGzgKAJEiTsxcItS2WYknsOGOpLXAV_6JWC9nMiurpeNeaSFTbx-Rc-2dVHb_3VzTVKQOPWdlKAHWDdPT5VImy2fuEJhwpRrky-YF_47GdB8dKDB8iuZq8IjDSCNTBxzFS1B_YTDHjViAh3oE8bJEQr92ZNd0LTErBFI548bVy6lLkC7ZwnJBUTSClBN-fDbmeyCZQpUO5XP2f_1bFY5KjLfV19VXi-V3aRaxX7M1JBcrLkun_zhZMpI6WWtw3DMbtHAotqlfu2yd2illUK1BVsFC4&refId=wvdqD6CcMpnov0h9eTomXA%3D%3D&trackingId=b%2FiO2B4dyI0VNQkvNiJmQQ%3D%3D&trk=flagship3_search_srp_jobs', 'March 18, 2024']
['3823523203', 'AEM Developer', 'Tata Consultancy Services', '', '', 'https://www.linkedin.com/jobs/view/3823523203/?eBP=CwEAAAGOUz6zWQtAhAFjMAmmWLFLvPnMwtJNVbUWLNi9IniVXT_c3uZ-QSCS7gNBXT3YJNigHfFa-IywehbPNa5OKaqc09a-rCfCeRLEl1xrH17d_oijEdSHOtNPec2avL8yOaB97rZhHUmX9JE0ZowiGX-dNrhhJJ6b_YmSYQGzuNtSVJgK8NwIVKQe9FC_j_NxJwTaT1CPuWYnUne8UByndQQ3E